In [1]:
# define a function to compute sentiments of the received tweets
from pyspark.ml import PipelineModel
pipelineFit=PipelineModel.load("preprocessingAndLR")
pipelineFitBBC=PipelineModel.load("preprocessingAndCategory")


22/05/02 11:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 11:54:45 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/05/02 11:54:45 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [2]:
print(pipelineFit)

PipelineModel_f3cba0012413


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, explode, when
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import time

if __name__ == "__main__":
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

    # read the tweet data from socket
    tweet_daf = spark \
        .readStream \
        .format("socket") \
        .option("host", "0.0.0.0") \
        .option("port", 5555) \
        .load()

    # type cast the column value
    tweet_df = tweet_daf.select(explode(split(col("value"), "TWEET__AN,")).alias("value"))
    tweet_df_string = (tweet_df.withColumn('tweet', split(tweet_df['value'], '____SEP,').getItem(0)).withColumn('location', split(tweet_df['value'], '____SEP,').getItem(1)).withColumn('favorites', split(tweet_df['value'], '____SEP,').getItem(2)).withColumn('followers', split(tweet_df['value'], '____SEP,').getItem(3)).withColumn('timestamp', split(tweet_df['value'], '____SEP,').getItem(4)).withColumn('name', split(tweet_df['value'], '____SEP,').getItem(5)))
    
    tweet_df_string =tweet_df_string.drop("value")
    tweet_df_sentiment = pipelineFit.transform(tweet_df_string).drop("tokens","filtered_words","vector","rawPrediction","probability")
    tweet_df_sentiment=tweet_df_sentiment.withColumnRenamed("prediction","sentiment")
    tweet_df_fl = pipelineFitBBC.transform(tweet_df_sentiment).drop("tokens","filtered_words","vector","rawPrediction","probability","cf")
    tweet_df_fl = tweet_df_fl.withColumn("favorites",col("favorites").cast("int")+1)
    tweet_df_fl = tweet_df_fl.withColumn("followers",col("followers").cast("int"))
    tweet_df_fl = tweet_df_fl.withColumn("prediction", when(tweet_df_fl.prediction == 0,"tech") \
          .when(tweet_df_fl.prediction == 1,"business").when(tweet_df_fl.prediction == 2,"politics").when(tweet_df_fl.prediction == 3,"sport").when(tweet_df_fl.prediction == 4,"entertainment").otherwise("Null"))
    tweet_df_fl = tweet_df_fl.withColumn("sentiment", when(tweet_df_fl.sentiment == 0.0,"Negative") .when(tweet_df_fl.sentiment == 4.0,"Positive").otherwise("Null"))
    
    # write the above data into memory. consider the entire analysis in all iteration (output mode = complete). and let the trigger runs in every 2 secs.
    writeTweet = tweet_df_fl.writeStream.format("memory").outputMode("append").queryName("processed").trigger(processingTime ='30 seconds').start()
    print("----- streaming is running -------")


22/05/02 11:59:59 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


----- streaming is running -------


22/05/02 12:00:01 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/87/_c2ttbjn7tl1w8dzqfn6nz7w0000gn/T/temporary-6b907440-72f1-436e-8527-f007e9243295. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [15]:
spark.stop()

In [10]:
tweets = spark.sql("select * from processed where tweet!=''")

In [11]:
tweets.count()

992

In [12]:
tweets.head(10)

[Row(tweet='“It is my great honor to be speaking tonight at the nation’s most distinguished superspreader event,” Trevor Noah joked at the top of his set at the White House Correspondents’ Dinner. https://t.co/gzfSpSxKQF', location='El Segundo, CA', favorites=1, followers=3925662, timestamp='Mon May 02 17:00:01 +0000 2022', name='Los Angeles Times', sentiment='Positive', prediction='politics'),
 Row(tweet='When @UR_Med\xa0faculty created https://t.co/TGf3mZJiEG to aid the people of #Ukraine, one primary care physician wanted to do more than simply donate. ', location=None, favorites=None, followers=None, timestamp=None, name=None, sentiment='Negative', prediction='business'),
 Row(tweet='If only @theallinpod would ever challenge his BS', location='United States', favorites=1, followers=58, timestamp='Mon May 02 17:00:01 +0000 2022', name='WallyO14', sentiment='Negative', prediction='sport'),
 Row(tweet="What Trevor Noah Got Desperately Wrong at the White House Correspondent's Dinner --

In [ ]:
%pylab inline
import plotly.express as px

In [13]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objects as go

import plotly.express as px
PORT = 5554
data = spark.sql("select * from processed").groupBy("sentiment").count().toPandas()

data.info()
#data.info()
app = dash.Dash(__name__)
app.layout = html.Div(
    children=[
        html.H1(children="LIVE DASHBOARD",),
        html.H3(children="Top 3 Tweets From Most Influenced People on live",)
        , dcc.Interval('userwithtopfollwers-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='userwithtopfollwers-graph'),
        html.H3(children="Top 3 liked Tweets on live",),
        dcc.Interval('tweetwithtoplikes-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='tweetwithtoplikes-graph'),
        
        html.H3(children="Overall Positive and Negative tweets classification",),
        dcc.Interval('sentiment-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='sentiment-graph'),
        
        html.H3(children="Overall Tweets in each Category",),
        dcc.Interval('category-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='category-graph'),
        
        html.H3(children="Overall Positive and Negative Tweets in each Category",),
        dcc.Interval('sentiment-category-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='sentiment-category-graph'),
        
        html.H3(children="Total likes for Tweets based on Sentiment",),
        dcc.Interval('weight-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='weight-graph'),
       
        
    ]
)
@app.callback(
        dash.dependencies.Output('sentiment-graph','figure'),
        [dash.dependencies.Input('sentiment-graph-update', 'n_intervals')])
def display_structure(n):
    df = spark.sql("select * from processed where tweet!=''").groupBy("sentiment").count().toPandas()
    fig = px.bar(df, x="sentiment", y="count")
    return fig

@app.callback(
        dash.dependencies.Output('category-graph','figure'),
        [dash.dependencies.Input('category-graph-update', 'n_intervals')])
def display_structure_2(n):
    df = spark.sql("select * from processed where tweet!=''").groupBy("prediction").count().toPandas()
    fig = px.pie(df, values='count', names='prediction',)
    return fig

@app.callback(
        dash.dependencies.Output('sentiment-category-graph','figure'),
        [dash.dependencies.Input('sentiment-category-graph-update', 'n_intervals')])
def display_structure_3(n):
    df = spark.sql("select * from processed where tweet!=''").groupBy("sentiment","prediction").count().toPandas()
    fig = px.bar(df, x="prediction", y="count", color="sentiment",title="Overall Positive and Negative Tweets in each Category")
    return fig

@app.callback(
        dash.dependencies.Output('weight-graph','figure'),
        [dash.dependencies.Input('weight-graph-update', 'n_intervals')])
def display_structure_4(n):
    df = spark.sql("select sentiment,count(favorites) as count from processed where tweet!='' group by sentiment").toPandas()
    #df = df.groupby("sentiment").favorites.sum()
    #fig = px.bar(df, x="favorites", y="followers")
    fig = px.bar(df,x="sentiment",y="count", color="sentiment", title="Total likes for Tweets based on Sentiment")
    return fig

@app.callback(
        dash.dependencies.Output('userwithtopfollwers-graph','figure'),
        [dash.dependencies.Input('userwithtopfollwers-graph-update', 'n_intervals')])
def display_structure_5(n):
    df = tweetPandas=spark.sql("select tweet,name,sentiment,followers from processed where tweet!='' order by followers desc limit 3").toPandas()
    #df = df.groupby("sentiment").favorites.sum()
    #fig = px.bar(df, x="favorites", y="followers")
    fig = go.Figure(data=[go.Table(header=dict(values=list(df.columns),fill_color='paleturquoise',align='left'),cells=dict(values=[df.tweet, df.name, df.sentiment, df.followers],fill_color='lavender',align='left'))])
    return fig

@app.callback(
        dash.dependencies.Output('tweetwithtoplikes-graph','figure'),
        [dash.dependencies.Input('tweetwithtoplikes-graph-update', 'n_intervals')])
def display_structure_6(n):
    df = tweetPandas=spark.sql("select tweet,name,sentiment,favorites from processed where tweet!='' order by favorites desc limit 3").toPandas()
    #df = df.groupby("sentiment").favorites.sum()
    #fig = px.bar(df, x="favorites", y="followers")
    fig = go.Figure(data=[go.Table(header=dict(values=list(df.columns),fill_color='paleturquoise',align='left'),cells=dict(values=[df.tweet, df.name, df.sentiment, df.favorites],fill_color='lavender',align='left'))])
    return fig





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  2 non-null      object
 1   count      2 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


In [14]:
if __name__ == '__main__':
    app.run_server(
        port=PORT)

Dash is running on http://127.0.0.1:5554/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5554/ (Press CTRL+C to quit)
127.0.0.1 - - [02/May/2022 12:01:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:01:18] "POST /_dash-update-component

127.0.0.1 - - [02/May/2022 12:05:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 12:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.